Hi all,

This is my first notebook. I've studied a few notebooks on this dataset. I learned a lot from this particular notebook https://www.kaggle.com/janiobachmann/credit-fraud-dealing-with-imbalanced-datasets written by Janio Martinez. Especially, I learned that you need to first split train/test, and then do undersampling or over sampling.

In this notebook I'll do the following:

1. Load and scale the data (there is enough visualization study, I'll ignore that). I know from other studies that the data is clean.
2. Split the data.
3. Train a VAE only on the fraud cases of the training data
4. use VAE to generate as many data necessary to match non-fraud case. build an augmented data
5. train an ensemble classifier
6. test on out-of-sample using different metrics
7. wait for comments and critics.



Import a few packages, classifiers, metrics and etc.

In [1]:
from os.path import exists
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, average_precision_score, precision_recall_curve
import numpy as np
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score
import pickle
import matplotlib.pyplot as plt

I like to put everything in a function for easier handling of data. I also make sure that the output is treated uniformly across experiments.

Here is what I'm doing in this function:
1. load the data
2. scale using StandardScaler in sklearn
3. go through each column except the 'Class' and scale it. Drop the column and replace it with a new name that has a prefix 'scaled_'
4. Put everything in the X data except 'Class' column that goes to y
5. split data
6. return as dataframe

In [2]:
def get_scaled_data_splitted():
    df = pd.read_csv('/kaggle/input/creditcardfraud/creditcard.csv')
    std_scaler = StandardScaler()
    columns = df.columns.drop(['Class'])
    for col in columns:
        df['scaled_' + str(col)] = std_scaler.fit_transform(df[col].values.reshape(-1, 1))
        df.drop(col, axis=1, inplace=True)
    X = df.drop('Class', axis=1)
    y = df['Class']
    xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2)
    return pd.concat((xtrain, ytrain), axis=1), pd.concat((xtest, ytest), axis=1)

A simple function to return the X value of the fraud case for train and test data

In [3]:
def fraud_or_not(dftrain, dftest):
    fraud = dftrain.loc[dftrain['Class'] == 1].sample(frac=1)
    fraudtest = dftest.loc[dftest['Class'] == 1].sample(frac=1)
    return fraud.drop('Class', axis=1), fraudtest.drop('Class', axis=1)

Before we go any further, we want to define our VAE class and a training function for the VAE. Read the inline comments.

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import InputLayer, Dense, Flatten, Reshape
from tensorflow.keras import Sequential
import time
import numpy as np

# This code is taken from Tensorflow tutorial on VAEs. It's turned from a CNN to a simple neuralnet which is more appropriate for our case here.

# almost the standard for activation these days
relu = tf.nn.relu


class VAE(tf.keras.Model):
    def __init__(self, ndim, latent_dim):
        super(VAE, self).__init__()
        self.latent_dim = latent_dim  # number of latent variables of the VAE
        
        # a simple neural net with one hidden layer. At the end we produce twice the number of latent variables because we are modeling the mean and variance of the Gaussian random variates.
        self.inference_net = Sequential(
            [
                InputLayer(input_shape=(ndim,)),
                Dense(100, activation=relu),
                Dense(2 * latent_dim)
            ]
        )
        
        # A simple decoder network at produces the mean of the variables. An improvement would be to model the variance as well. The training becomes tricky since the variance can collapse.
        self.generative_net = Sequential(
            [
                InputLayer(input_shape=(latent_dim,)),
                Dense(100, activation=relu),
                Dense(ndim)
            ])

    # We need this function to generate samples. Start with random normal noise. pass it through the decoder, and you get your samples
    @tf.function
    def sample(self, num_samples=100, eps=None):
        if eps is None:
            eps = tf.random.normal(shape=(num_samples, self.latent_dim))
        return self.decode(eps)

    # a function to call the encoder network
    def encode(self, x):
        mean, logvar = tf.split(self.inference_net(x), num_or_size_splits=2, axis=1)
        return mean, logvar

    # a function to reparameterie the encoder for the ease of backpropagation
    def reparameterize(self, mean, logvar):
        eps = tf.random.normal(shape=mean.shape)
        return eps * tf.exp(logvar * 0.5) + mean

    # a function to call the decoder
    def decode(self, z):
        return self.generative_net(z)


In [5]:
# a function for computing the KL term of Gaussian distribution
def log_normal_pdf(sample, mean, logvar, raxis=1):
    log2pi = tf.math.log(2. * np.pi)
    return tf.reduce_sum(-0.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi),
                         axis=raxis)

# a function to compute the loss of the VAE
@tf.function
def compute_loss(model, x):
    mean, logvar = model.encode(x)
    logvar = tf.clip_by_value(logvar, -88., 88.)
    z = model.reparameterize(mean, logvar)
    xmean = model.decode(z)
    logpx_z = -tf.reduce_sum((x - xmean) ** 2, axis=1)  # ad-hoc l2 loss that is pretty close to log-prob of a gaussian distribution withtout taking into account the variance
    logpz = log_normal_pdf(z, 0.0, 0.0)
    logqz_x = log_normal_pdf(z, mean, logvar)
    return -tf.reduce_mean(logpx_z + logpz - logqz_x)

# A function that given the model computes the loss, the gradients and apply the parameter update
@tf.function
def compute_apply_gradients(model, x, optimizer):
    with tf.GradientTape() as tape:
        loss = compute_loss(model, x)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

The training function is easy and self explanatory. I added a few lines to make sure that I can re-load the model every time we want to.

In [6]:
def train(xtrain, xtest, model=None, load=False, filepath=None):
    optimizer = tf.keras.optimizers.Adam(1e-3)
    epochs = 2000
    latent_dim = 20
    num_train, ndim = xtrain.shape
    num_test, _ = xtest.shape
    if model is None:
        model = VAE(ndim, latent_dim)
    if load and filepath is not None:
        model.load_weights(filepath=filepath)
        return model
    else:
        batch_size = 32
        train_dataset = tf.data.Dataset.from_tensor_slices(xtrain.values.astype(np.float32)).shuffle(num_train).batch(
            batch_size)

        test_dataset = tf.data.Dataset.from_tensor_slices(xtest.values.astype(np.float32)).shuffle(num_test).batch(num_test)

        for epoch in range(1, epochs + 1):
            start_time = time.time()
            for train_x in train_dataset:
                compute_apply_gradients(model, train_x, optimizer)
            end_time = time.time()

            if epoch % 100 == 0:
                loss = tf.keras.metrics.Mean()
                for test_x in test_dataset:
                    loss(compute_loss(model, test_x))
                elbo = -loss.result()
                print('Epoch: {}, Test set psudo-ELBO: {}, '
                      'time elapse for current epoch {}'.format(epoch, elbo, end_time - start_time))
                model.save_weights('saved_models/model_%d_at_%d' % (latent_dim, epoch))
    return model

Once we have the VAE trained, we can pass it to this function and samples from it to increase the number of fraud cases.

In [7]:
def augment_data(data, model):
    num_samples = data['Class'].value_counts()[0] - data['Class'].value_counts()[1]
    samples = model.sample(num_samples=num_samples).numpy()
    dfnew = pd.DataFrame(samples, columns=data.columns.drop('Class'))
    dfnew['Class'] = np.ones(len(samples), dtype=np.int)
    dfnew = pd.concat((data, dfnew), ignore_index=True).sample(frac=1)
    return dfnew

In [8]:
# get the data and split it
dtrain, dtest = get_scaled_data_splitted()
# get the fraud cases from train and test
Xf, Xft = fraud_or_not(dtrain, dtest)
# get the traied VAE model
model = train(Xf, Xft, load=False, filepath='saved_models/model_20_at_1900')
# augment the data using the VAE model
augmented = augment_data(dtrain, model)
X = augmented.drop('Class', axis=1)
y = augmented['Class']
Xt = dtest.drop('Class', axis=1)
yt = dtest['Class']

Epoch: 100, Test set psudo-ELBO: -50.271484375, time elapse for current epoch 0.022028684616088867
Epoch: 200, Test set psudo-ELBO: -40.77564239501953, time elapse for current epoch 0.02196335792541504
Epoch: 300, Test set psudo-ELBO: -38.007659912109375, time elapse for current epoch 0.02331399917602539
Epoch: 400, Test set psudo-ELBO: -33.243804931640625, time elapse for current epoch 0.023506879806518555
Epoch: 500, Test set psudo-ELBO: -31.718900680541992, time elapse for current epoch 0.022860288619995117
Epoch: 600, Test set psudo-ELBO: -30.620738983154297, time elapse for current epoch 0.023627042770385742
Epoch: 700, Test set psudo-ELBO: -32.56962966918945, time elapse for current epoch 0.020328044891357422
Epoch: 800, Test set psudo-ELBO: -29.3978328704834, time elapse for current epoch 0.0248868465423584
Epoch: 900, Test set psudo-ELBO: -29.8550968170166, time elapse for current epoch 0.020946979522705078
Epoch: 1000, Test set psudo-ELBO: -29.949703216552734, time elapse for 

Now that we have the augmented X, y, we are going to train a classifier. Then we going to test it on the held-out test set. Be careful that if you're loading a VAE model, it may contain your test set becase the model is train on a specific train/test split, but upon loading you might get a different split.

In [9]:
classifiers = {
    "XGBClassifier": XGBClassifier(),
    "BaggingClassifier": BaggingClassifier(),
    "RandomForestClassifier": RandomForestClassifier(),
}

N = 10000
for key, classifier in classifiers.items():
    print('_' * 50)
    name = key
    print(name)
    if exists(name):
        print('loading...')
        classifier = pickle.load(open(name, 'rb'))
        training_score = cross_val_score(classifier, X[:N], y[:N], cv=5)
        print("Classifiers: ", name, "Has a training score of", round(training_score.mean(), 2) * 100, "% accuracy score")
    else:
        classifier.fit(X, y)
        print('validating ...')
        training_score = cross_val_score(classifier, X[:N], y[:N], cv=5)
        print("Classifiers: ", name, "Has a training score of", round(training_score.mean(), 2) * 100, "% accuracy score")
        pickle.dump(classifier, open(name, 'wb'))
    print(classifier.score(X, y))
    print(classifier.score(Xt, yt))
    y_pred = classifier.predict(Xt)
    cm = confusion_matrix(yt, y_pred)
    print(cm)
    print(classification_report(yt, y_pred))
    print(average_precision_score(yt, y_pred))


__________________________________________________
XGBClassifier
validating ...
Classifiers:  XGBClassifier Has a training score of 99.0 % accuracy score
0.9999076667384814
0.9988588883817282
[[56824    54]
 [   11    73]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56878
           1       0.57      0.87      0.69        84

    accuracy                           1.00     56962
   macro avg       0.79      0.93      0.85     56962
weighted avg       1.00      1.00      1.00     56962

0.499724419783365
__________________________________________________
BaggingClassifier
validating ...
Classifiers:  BaggingClassifier Has a training score of 98.0 % accuracy score
0.9996130796660174
0.9968224430321969
[[56708   170]
 [   11    73]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56878
           1       0.30      0.87      0.45        84

    accuracy                         